In [4]:
import numpy as np
import cv2
from primesense import openni2
from primesense import _openni2 as c_api

In [5]:
def callback(x):
    pass

ilowH = 0
ihighH = 179

ilowS = 0
ihighS = 255
ilowV = 0
ihighV = 255

#Camera setting
image_size_x = 480  
image_size_y = 640  
fps = 30
"""
image_size_x =1280
image_size_y =960
fps =30
"""

image_centre_x = image_size_x // 2
image_centre_y = image_size_y // 2


default_w = 225
default_h = 225

default_pos_x = 225
default_pos_y = 200

kernel = np.ones((5,5),np.uint8)

##Path to rgb save directory
path_object = "save_files/objects/chips/Cam1/"

In [ ]:
## Path of the OpenNI redistribution OpenNI2.so
dist ='AstraSDK/lib/Plugins/openni2'

## initialize openni and check
openni2.initialize(dist) #'C:\Program Files\OpenNI2\Redist\OpenNI2.dll') # accepts the path of the OpenNI redistribution
if (openni2.is_initialized()):
    print ("openNI2 initialized")
else:
    print ("openNI2 not initialized")

## Register the device
dev = openni2.Device.open_any()

## create the streams stream
rgb_stream = dev.create_color_stream()
#rgb_stream.set_video_mode(c_api.OniVideoMode(pixelFormat = c_api.OniPixelFormat.ONI_PIXEL_FORMAT_RGB888, resolutionX = image_size_x, resolutionY = image_size_y, fps = fps))
#rgb_stream.set_property(c_api.ONI_STREAM_PROPERTY_AUTO_EXPOSURE, False)
## Check and configure the mirroring -- default is True
rgb_stream.set_mirroring_enabled(False)

## start the stream
rgb_stream.start()

def get_rgb():
    """
    Returns numpy 3L ndarray to represent the rgb image.
    """
    bgr   = np.fromstring(rgb_stream.read_frame().get_buffer_as_uint8(),dtype=np.uint8).reshape(image_size_x, image_size_y,3)
    rgb   = cv2.cvtColor(bgr,cv2.COLOR_BGR2RGB)
    return rgb    
#get_rgb

cv2.namedWindow('image',cv2.WINDOW_NORMAL)
cv2.resizeWindow('image', 1400,800)
# create trackbars for color change
cv2.createTrackbar('lowH','image',ilowH,360,callback)
cv2.createTrackbar('highH','image',ihighH,360,callback)

cv2.createTrackbar('lowS','image',ilowS,255,callback)
cv2.createTrackbar('highS','image',ihighS,255,callback)

cv2.createTrackbar('lowV','image',ilowV,255,callback)
cv2.createTrackbar('highV','image',ihighV,255,callback)

cv2.createTrackbar('Xpos','image',default_pos_x,image_size_x,callback)
cv2.createTrackbar('Ypos','image',default_pos_y,image_size_y,callback)

cv2.createTrackbar('Xcropping','image',default_w,image_size_x,callback)
cv2.createTrackbar('Ycropping','image',default_h,image_size_y,callback)

## main loop
s=0
done = False
start_saving = False
while not done:
    key = cv2.waitKey(1) & 255
    ## Read keystrokes
    if key == 27: # terminate
        print ("\tESC key detected!")
        done = True
    elif chr(key) =='s': #screen capture
        start_saving = True
        print ("\ts key detected. Saving image and distance map {}".format(s))
    if (start_saving):
        cv2.imwrite("" + path_object + "extracted/rgb_"+str(s)+'.png', rgb_result)
        cv2.imwrite("" + path_object + "mask/mask_"+str(s)+'.png', mask)
        cv2.imwrite("" + path_object + "result/result_"+str(s)+'.png', result)
        cv2.imwrite("" + path_object + "image_original/original_"+str(s)+'.png', rgb)
        s+=1
    #if
    
    ## Streams
    #RGB
    rgb = get_rgb()
    original_rgb = rgb.copy()
    
    #rgb = cv2.GaussianBlur(rgb, (5, 5), 0) # Optional, remove some noise can help when finding contours
    
    ##HSV cropping
    # get trackbar positions
    ilowH = cv2.getTrackbarPos('lowH', 'image')
    ihighH = cv2.getTrackbarPos('highH', 'image')
    ilowS = cv2.getTrackbarPos('lowS', 'image')
    ihighS = cv2.getTrackbarPos('highS', 'image')
    ilowV = cv2.getTrackbarPos('lowV', 'image')
    ihighV = cv2.getTrackbarPos('highV', 'image')
    
    xpos = cv2.getTrackbarPos('Xpos', 'image')
    ypos = cv2.getTrackbarPos('Ypos', 'image')
    
    xcrop = cv2.getTrackbarPos('Xcropping', 'image')
    ycrop = cv2.getTrackbarPos('Ycropping', 'image')

    hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)
    lower_hsv = np.array([ilowH, ilowS, ilowV])
    higher_hsv = np.array([ihighH, ihighS, ihighV])
    mask = cv2.inRange(hsv, lower_hsv, higher_hsv)
    mask_inv = cv2.bitwise_not(mask)

    rgb_visu = cv2.bitwise_and(rgb, rgb, mask=mask)
    rgb_cropped = rgb_visu.copy()
    cv2.rectangle(rgb_visu, (xpos, ypos), (xpos+xcrop, ypos+ycrop), (0, 255, 0), 1);
    
    mask_rec = np.zeros(rgb_cropped.shape[:2],np.uint8)
    mask_rec[ypos:ypos+ycrop,xpos:xpos+xcrop] = 255
    rgb_masked = cv2.bitwise_and(rgb_cropped,rgb_cropped,mask = mask_rec)
    # canvas
    canvas = np.hstack((rgb,rgb_visu,rgb_masked))

    ## Display the stream
    cv2.imshow('image', canvas )
    try :
        rgb_cropped = rgb_visu[ypos:ypos+ycrop,xpos:xpos+xcrop]
        rgb_result = original_rgb[ypos:ypos+ycrop,xpos:xpos+xcrop]
        gray = cv2.cvtColor(rgb_cropped, cv2.COLOR_RGB2GRAY)
        ret,thresh = cv2.threshold(gray,1,255,0)
        thresh = cv2.dilate(thresh,kernel,iterations = 1) #Optional
        thresh = cv2.erode(thresh,kernel,iterations = 2) #Optional
        image, contours, heirarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        cnt = max(contours, key = cv2.contourArea)
        cv2.drawContours(rgb_cropped, [cnt], 0, (0,255,0),2, cv2.LINE_AA, maxLevel=1)

        cv2.imshow('test',rgb_cropped)
        cv2.imshow('thresh',thresh)

        refined_mask = np.zeros(rgb_cropped.shape,dtype='uint8')
        refined_mask = cv2.drawContours(refined_mask, [cnt], -1, (255 , 255 , 255),thickness=cv2.FILLED)
        #refined_mask = cv2.bitwise_not(refined_mask)
        img2gray = cv2.cvtColor(refined_mask,cv2.COLOR_BGR2GRAY)
        ret, mask = cv2.threshold(img2gray, 1, 255, cv2.THRESH_BINARY)
        result= cv2.bitwise_and(rgb_result,rgb_result,mask=mask)
        cv2.imshow('result',result)
    except : 
        continue
# end while

## Release resources 
cv2.destroyAllWindows()
rgb_stream.stop()
openni2.unload()
print ("Terminated")

openNI2 initialized


/home/jeremy/.local/lib/python3.5/site-packages/ipykernel_launcher.py:28: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
